#  Sentiment analysis on twitter samples

Who does not know it ? Twitter is a social networking service owned  on which users post and interact with messages known as "tweets". 
In this notebook, we are going to train and develop a simple Twitter Sentiment Analysis supervised learning model using python and NLP libraries.
Let's start :) !

In [2]:
# importing necessary libraries

import nltk, re, string
from nltk.corpus import stopwords, twitter_samples
import numpy as np
import pickle
import pandas as pd

## preprecessing data

In [1]:
def process_tweet(tweet):
  #first we are going to remove morphological affixes from words, leaving only the word stem.
    stemmer = nltk.PorterStemmer()
    stopwords_english = stopwords.words('english')
    tweet = re.sub(r'\$\w*', '', tweet)
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    tweet = re.sub(r'#', '', tweet)

    
    tokenizer = nltk.TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)#) with thismethod we are able to convert the stream 
    #of words into small  tokens so that we can analyse the audio stream with the help of nltk

    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and
                word not in string.punctuation):
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)

    return tweets_clean


In [4]:
def dict_freqs(tweets, ys):
    # Convert np array to list since the function zip needs an iterable.
    yslist = np.squeeze(ys).tolist()

    # Start with an empty dictionary and populate it by looping over all tweets
    # and over all processed words in each tweet.
    freqs = {}
    # zip function takes in iterables as arguments and returns an iterator
    for y, tweet in zip(yslist, tweets):
        for word in process_tweet(tweet): #process_tweet returns a a list of clean tweets
            pair = (word, y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1

    return freqs

In [7]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
# let's test our function build_freq

tweets = ['i am brilliant', 'i am depressed', 'i am sad', 'i am tired', 'i am tired']
ys = [1, 0, 0, 0, 0]
res = dict_freqs(tweets, ys)
print(res)

{('brilliant', 1): 1, ('depress', 0): 1, ('sad', 0): 1, ('tire', 0): 2}


In [11]:
nltk.download('twitter_samples')

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.


True

In [12]:
# select the set of positive and negative tweets
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

In [14]:
type(all_positive_tweets)

list

## splitting data (train&test)

In [16]:
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

In [18]:
#concating the pos and neg tweets for train ant test parts
train_x = train_pos + train_neg
test_x = test_pos + test_neg

In [19]:
# combine positive and negative labels
# We are building our y - target variable here
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

In [21]:
# create frequency dictionary
freqs = dict_freqs(train_x, train_y)

In [22]:
# check out the output

print("type(freqs) = " + str(type(freqs)))
print("len(freqs) = " + str(len(freqs.keys())))

type(freqs) = <class 'dict'>
len(freqs) = 11338


In [23]:
# test the function below

print('This is an example of a positive tweet: \n', train_x[20])
print('\nThis is  the processed version of our example: \n', process_tweet(train_x[20]))

This is an example of a positive tweet: 
 #FollowFriday @MBandScott_ @Eric_FLE @pointsolutions3 for being top new followers in my community this week :)

This is  the processed version of our example: 
 ['followfriday', 'top', 'new', 'follow', 'commun', 'week', ':)']


## Logistic regression from scratch

In [27]:
def sigmoid(z):
    zz = np.negative(z)
    h = 1 / (1 + np.exp(zz))
    return h

In [28]:
# Cost function and Gradient
def gradientDescent(x, y, theta, alpha, num_iters):
    """
    Input:
        x: matrix of features which is (m,n+1)
        y: corresponding labels of the input matrix x, dimensions (m,1)
        theta: weight vector of dimension (n+1,1)
        alpha: learning rate
        num_iters: number of iterations you want to train your model for
    Output:
        J: the final cost
        theta: your final weight vector
    We will print the cost to make sure that it is going down.
    """
    # get 'm', the number of rows in matrix x
    m = x.shape[0]
    for i in range(0, num_iters):
        z = np.dot(x, theta)
        h = sigmoid(z)
        # calculate the cost function
        cost = -1. / m * (np.dot(y.transpose(), np.log(h)) + np.dot((1 - y).transpose(), np.log(1 - h)))
        # update the weights theta
        theta = theta - (alpha / m) * np.dot(x.transpose(), (h - y))

    cost = float(cost)
    return cost, theta


In [29]:
#  Extracting the features

def extract_features(tweet, freqs):
    """
    Input:
        tweet: a list of words for one tweet
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
    Output:
        x: a feature vector of dimension (1,3) which are the text and the number of positive and negative words
    """

    word_l = process_tweet(tweet)
    x = np.zeros((1, 3))

    # bias term is set to 1
    x[0, 0] = 1

    for word in word_l:
        # increment the word count for the positive label 1
        x[0, 1] += freqs.get((word, 1.0), 0)
        # increment the word count for the negative label 0
        x[0, 2] += freqs.get((word, 0.0), 0)

    assert (x.shape == (1, 3))
    return x

In [32]:
# test on training data

coeff = extract_features(train_x[22], freqs)
print(coeff)

[[1.000e+00 3.006e+03 1.240e+02]]


How these coefficients will be used to predict in Logistic Regression?

First a hypothesis is build which for our case will be h(x) = b1 + b2*x1 + b3*x2

here b1 = 1, b2 and b3 are determined by cost and gradient function, x1 and x2 are the positive and negative words feature set.

In [34]:
# Training Your Model

# collect the features 'x' and stack them into a matrix 'X'
X = np.zeros((len(train_x), 3))
for i in range(len(train_x)):
    X[i, :] = extract_features(train_x[i], freqs)

# training labels corresponding to X
Y = train_y

# Apply gradient descent
# these values are predefined 
J, theta = gradientDescent(X, Y, np.zeros((3, 1)), 1e-9, 1500)

finally we have our model and we want to predict a tweet that we choose

In [35]:
def predict_tweet(tweet, freqs, theta):
    """
    Input:
        tweet: a string
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
        theta: (3,1) vector of weights
    Output:
        y_pred: the probability of a tweet being positive or negative
    """
    # extract the features of the tweet and store it into x
    x = extract_features(tweet, freqs)
    y_pred = sigmoid(np.dot(x, theta))

    return y_pred


In [36]:
def test_logistic_regression(test_x, test_y, freqs, theta):
    """
    Input:
        test_x: a list of tweets
        test_y: (m, 1) vector with the corresponding labels for the list of tweets
        freqs: a dictionary with the frequency of each pair (or tuple)
        theta: weight vector of dimension (3, 1)
    Output:
        accuracy: (number of tweets classified correctly) / (total number of tweets)
    """
    # the list for storing predictions
    y_hat = []

    for tweet in test_x:
        # get the label prediction for the tweet
        y_pred = predict_tweet(tweet, freqs, theta)
        if y_pred > 0.5:
            y_hat.append(1)
        else:
            y_hat.append(0)

    accuracy = (y_hat == np.squeeze(test_y)).sum() / len(test_x)

    return accuracy

In [37]:
tmp_accuracy = test_logistic_regression(test_x, test_y, freqs, theta)
print(f"Logistic regression model's accuracy = {tmp_accuracy:.4f}")

Logistic regression model's accuracy = 0.9950


In [39]:
# Predict with our own tweet

def predict(sentence):
    yhat = predict_tweet(sentence, freqs, theta)
    if yhat > 0.5:
        return 'Positive sentiment'
    elif yhat == 0:
        return 'Neutral sentiment'
    else:
        return 'Negative sentiment'

In [40]:
my_tweet = 'I am very happy that I succeded to create this model. I hope you enjoyed it and it wasnt so tiring '

res = predict(my_tweet)
print(res)

Positive sentiment
